In [18]:
import pandas as pd
from tqdm import tqdm
import os

champion control score unit is seconds per minute

In [19]:
'''
1.champion control score unit is seconds per minute
2.Made with the average of all games, there is no division between the pre, middle or late stages
'''

'\n1.champion control score unit is seconds per minute\n2.Made with the average of all games, there is no division between the pre, middle or late stages\n'

In [20]:
df_train_pre = pd.read_csv("../datasets/train_pre.csv")
df_train_mid = pd.read_csv("../datasets/train_mid.csv")
df_train_late = pd.read_csv("../datasets/train_late.csv")
df_train = pd.read_csv("../datasets/processed_data/train.csv")
championID = pd.read_csv("../datasets/championID.csv")

In [4]:
def attack_defense_score(df):
    dfs = []
    for i in range(1,11):
        df_tmp = df.loc[:,['gameDuration','Player_'+str(i)+'_pick','Player_'+str(i)+'_dmg_dealt','Player_'+str(i)+'_dmg_taken']]
        df_tmp['atk'] = df_tmp['Player_'+str(i)+'_dmg_dealt']/df['gameDuration']*60
        df_tmp['def'] = df_tmp['Player_'+str(i)+'_dmg_taken']/df['gameDuration']*60

        df_sum = df_tmp.groupby(['Player_'+str(i)+'_pick']).size().reset_index(name='sum'+str(i))
        df_atk = df_tmp.groupby(['Player_'+str(i)+'_pick'])['atk'].sum().reset_index(name='atk'+str(i))
        df_def = df_tmp.groupby(['Player_'+str(i)+'_pick'])['def'].sum().reset_index(name='def'+str(i))

        atk_def = df_sum.merge(df_atk, on = ['Player_'+str(i)+'_pick'])
        atk_def = atk_def.merge(df_def, on = ['Player_'+str(i)+'_pick'])

        atk_def.rename(columns = {"Player_"+str(i)+"_pick":'id'},inplace=True)
        dfs.append(atk_def)

    ads = championID.merge(dfs[0],on='id',how = 'left')
    for i in range(1,10):
        ads = ads.merge(dfs[i],on='id',how = 'left')
    ads.fillna(0,inplace=True)
    
    ads['sum'] = ads['sum1']+ads['sum2']+ads['sum3']+ads['sum4']+ads['sum5']+ads['sum6']+ads['sum7']+ads['sum8']+ads['sum9']+ads['sum10'] 
    ads['atk'] = ads['atk1']+ads['atk2']+ads['atk3']+ads['atk4']+ads['atk5']+ads['atk6']+ads['atk7']+ads['atk8']+ads['atk9']+ads['atk10']
    ads['def'] = ads['def1']+ads['def2']+ads['def3']+ads['def4']+ads['def5']+ads['def6']+ads['def7']+ads['def8']+ads['def9']+ads['def10']  
    ads['attack_score'] = ads['atk']/ads['sum']
    ads['defense_score'] = ads['def']/ads['sum']
    championAttackDefenseScore = ads.loc[:,['id','name','attack_score','defense_score']]
    championAttackDefenseScore = championAttackDefenseScore.round(2)
    return championAttackDefenseScore
    

In [5]:
def control_score(df):
    dfs = []
    for i in range(1,11):
        df_tmp = df.loc[:,['gameDuration','Player_'+str(i)+'_pick','Player_'+str(i)+'_time_ccing']]
        df_tmp['control_score'] = df_tmp['Player_'+str(i)+'_time_ccing']/df['gameDuration']*60
        df_sum = df_tmp.groupby(['Player_'+str(i)+'_pick']).size().reset_index(name='sum'+str(i))
        df_con = df_tmp.groupby(['Player_'+str(i)+'_pick'])['control_score'].sum().reset_index(name='control_sum'+str(i))
        control = df_sum.merge(df_con, on = ['Player_'+str(i)+'_pick'])
        control.rename(columns = {"Player_"+str(i)+"_pick":'id'},inplace=True)
        dfs.append(control)

    cs = championID.merge(dfs[0],on='id',how = 'left')
    for i in range(1,10):
        cs = cs.merge(dfs[i],on='id',how = 'left')
    cs.fillna(0,inplace=True)
    
    cs['sum'] = cs['sum1']+cs['sum2']+cs['sum3']+cs['sum4']+cs['sum5']+cs['sum6']+cs['sum7']+cs['sum8']+cs['sum9']+cs['sum10'] 
    cs['control_sum'] = cs['control_sum1']+cs['control_sum2']+cs['control_sum3']+cs['control_sum4']+cs['control_sum5']+cs['control_sum6']+cs['control_sum7']+cs['control_sum8']+cs['control_sum9']+cs['control_sum10'] 
    cs['control_score'] = cs['control_sum']/cs['sum']
    
    championControlScore = cs.loc[:,['id','name','control_score']]
    championControlScore = championControlScore.round(4)
    return championControlScore
    


In [6]:
championAttackDefenseScore_pre = attack_defense_score(df_train_pre)
championAttackDefenseScore_mid = attack_defense_score(df_train_mid)
championAttackDefenseScore_late = attack_defense_score(df_train_late)

championAttackDefenseScore_pre.to_csv("../Useful Features/4.championAttackDefenseScore_pre.csv", index=False)
championAttackDefenseScore_mid.to_csv("../Useful Features/4.championAttackDefenseScore_mid.csv", index=False)
championAttackDefenseScore_late.to_csv("../Useful Features/4.championAttackDefenseScore_late.csv", index=False)

In [7]:
championControlScore = control_score(df_train)
championControlScore.to_csv("../Useful Features/5.championControlScore.csv", index = False)

In [24]:
def gold_ability(df):
    dfs = []
    for i in range(1,11):
        df_tmp = df.loc[:,['gameDuration','Player_'+str(i)+'_pick','Player_'+str(i)+'_gold']]
        df_tmp['gold_per_min'] = df_tmp['Player_'+str(i)+'_gold']/df['gameDuration'] * 60
        df_sum = df_tmp.groupby(['Player_'+str(i)+'_pick']).size().reset_index(name='sum'+str(i))
        df_g = df_tmp.groupby(['Player_'+str(i)+'_pick'])['gold_per_min'].sum().reset_index(name='gold_sum'+str(i))
        control = df_sum.merge(df_g, on = ['Player_'+str(i)+'_pick'])
        control.rename(columns = {"Player_"+str(i)+"_pick":'id'},inplace=True)
        dfs.append(control)

    cs = championID.merge(dfs[0],on='id',how = 'left')
    for i in range(1,10):
        cs = cs.merge(dfs[i],on='id',how = 'left')
    cs.fillna(0,inplace=True)
    
    cs['sum'] = cs['sum1']+cs['sum2']+cs['sum3']+cs['sum4']+cs['sum5']+cs['sum6']+cs['sum7']+cs['sum8']+cs['sum9']+cs['sum10'] 
    cs['gold_sum'] = cs['gold_sum1']+cs['gold_sum2']+cs['gold_sum3']+cs['gold_sum4']+cs['gold_sum5']+cs['gold_sum6']+cs['gold_sum7']+cs['gold_sum8']+cs['gold_sum9']+cs['gold_sum10']  
    # cs['gold_sum']  = 0
    # for i in range(1,11):
    #     cs['gold_sum'] += cs['gold_sum'+str(i)] * cs['sum' + str(i)]
    
    cs['gold_ability'] = cs['gold_sum'] / cs['sum']
    championGoldAbility = cs.loc[:,['id','name','gold_ability']]
    championGoldAbility = championGoldAbility.round(4)
    return championGoldAbility

In [28]:
championGoldAbility_pre = gold_ability(df_train_pre)
championGoldAbility_mid = gold_ability(df_train_mid)
championGoldAbility_late = gold_ability(df_train_late)

In [29]:
championGoldAbility_pre.to_csv("../Useful Features/6.championGoldAbility_pre1.csv", index=False)
championGoldAbility_mid.to_csv("../Useful Features/6.championGoldAbility_mid1.csv", index=False)
championGoldAbility_late.to_csv("../Useful Features/6.championGoldAbility_late1.csv", index=False)